In [1]:
from pdftotext import pdf_to_text
from rich.progress import Progress
from segmentation import naive_split
import re

import embed

pdf_path = "/run/media/pocketdoc/Storage/datasets/21ajfr.pdf"

# document = pdf_to_text(pdf_path, verbose=True)

# # Save the document to a file
# with open('document.txt', 'w') as f:
    # f.write(document)

# Load the document
with open('pdf_output/21ajfr/21ajfr.md', 'r') as f:
    document = f.read()


# Remove markdown image embeds
document = re.sub(r"!\[.*\]\(.*\)", '', document)

# Split the text
segments = naive_split(document, max_tokens=150, overlap=0.1)


# Embed the segments in batches
segment_embeddings = embed.batch_embed(segments, verbose=True, batch_size=4)


Output()

In [2]:
from sentence_transformers import CrossEncoder
import json
from structml import line_heal


# Load the model, here we use our base sized model
model = CrossEncoder("mixedbread-ai/mxbai-rerank-large-v1")

# Question
question = "Let me know if you have any tips for identifying the subtle differences between similar opaque material like jasper versus petrified wood."

# Embed the question
question_embedding = embed.embed(question, query=True)

similarity_scores = embed.embed_similarity(question_embedding, segment_embeddings)

# Create a list of dicts with the segment, similarity, and index
# segment_data = [{"segment": segment, "similarity": similarity, "index": index} for index, (segment, similarity) in enumerate(zip(segments, similarity_scores))]
# And the embeddings for the segments
segment_data = [{"segment": segment, "similarity": similarity, "index": index, "embedding": embedding} for index, (segment, similarity, embedding) in enumerate(zip(segments, similarity_scores, segment_embeddings))]

# Sort the segments by similarity
sorted_segments = sorted(segment_data, key=lambda x: x["similarity"], reverse=True)

# Get the top 50 segments
top_segments = sorted_segments[:50]

results = model.rank(question, [segment["segment"] for segment in top_segments])

# Sort by "corpus_id"
results = sorted(results, key=lambda x: x["corpus_id"])

# Add "score" to top_segments
for segment, result in zip(top_segments, results):
    segment["score"] = result["score"]


# Take all segments with a score greater than 0.5 times the max score
threshold = 0.5 * max([segment["score"] for segment in top_segments])
filtered_segments = [segment for segment in top_segments if segment["score"] > threshold]

# Remove duplicates
filtered_segments = list({segment["index"]: segment for segment in filtered_segments}.values())

# Take only enough segments to have at most 60000 characters
total_characters = 0
    

# Sort by "index"
filtered_segments = sorted(filtered_segments, key=lambda x: x["index"])

# create a list of the segments not in the filtered segments
# deduped_segment_data = [segment for segment in segment_data if segment["index"] not in [filtered_segment["index"] for filtered_segment in filtered_segments]]
deduped_segment_data = segment_data

deduped_segment_embeddings = [segment["embedding"] for segment in deduped_segment_data]

additions = []

# For each filtered segment, find the most similar segment in the deduped segments and add it to the filtered segments
for filtered_segment in filtered_segments:
    similarity_scores = embed.embed_similarity(filtered_segment["embedding"], deduped_segment_embeddings)
    # Add the similarity score to the deduped segment data
    for index, similarity_score in enumerate(similarity_scores):
        deduped_segment_data[index]["similarity"] = similarity_score

    # Sort the deduped segments by similarity
    deduped_segment_data = sorted(deduped_segment_data, key=lambda x: x["similarity"], reverse=True)

    # Add the most similar segment to additions
    additions.append(deduped_segment_data[0])

# Add the additions to the filtered segments
filtered_segments += additions

# Fetch the segments before and after each of the filtered segments from segment_data
final_segments = []

for segment in filtered_segments:
    index = segment["index"]
    final_segments.append(segment_data[index - 1])
    final_segments.append(segment_data[index])
    final_segments.append(segment_data[index + 1])

# Remove duplicates
final_segments = list({segment["index"]: segment for segment in final_segments}.values())

# Take only enough segments to have at most 60000 characters
total_characters = 0

for segment in final_segments:
    total_characters += len(segment["segment"])

    if total_characters > 40000:
        final_segments.remove(segment)
        total_characters -= len(segment["segment"])

# Sort by "index"
final_segments = sorted(final_segments, key=lambda x: x["index"])

# Prepare the output string
output = ""

for segment in final_segments:
    output += "Index: {}\n".format(segment["index"])
    output += "Usefulness: {}\n".format(segment["similarity"])
    output += segment["segment"]
    output += "\n\n"
    
# Save the output to notes.md
with open('notes.md', 'w') as f:
    f.write(output)
    
print(output)

Index: 68
Usefulness: 0.3867267370223999
```

# , Collecting Regulations

## * Minerals

```
The Bureau of Land Management, United States Forest Service, and the 
                                                                     
Department of Natural Resources consider amateur rockhounding an outdoor 
                                                                    
recreation. Just like fishing and other outdoor activities approved on public 
                                                                     
lands, rockhounding is not without its rules and regulations. Recreational 
                                                                    
noncommercial collecting is allowed using hand tools only. 
                                                     
   There are a few rockhounding sites located on public lands that are close 
                                                                     
to Wilderness Study Areas or Wilderness Area. While on BLM land you 